In [4]:
import pandas as pd
import os

In [5]:
# Fonction qui traite les csv fournit par Lepont
# -> Conversion aux bons formats etc..

def traitement_data(file):
    
    print("Chargement du csv ... ")
    df = pd.read_csv(file, keep_default_na=False)
    print('\n')

    print("Traitement des données ... ")
    print('\n')
    
    print('Conversion datetime :')
    df['ping_time'] = pd.to_datetime(df['ping_time'].str.strip(), format='%d/%m/%Y %H:%M:%S')
    
    df['duedate'] = pd.to_datetime(df['duedate'].str.strip().str.replace('T', ' ')\
                                                .str.replace('+02', '' ).str.replace('+01', '' ), format='%Y-%m-%d %H:%M:%S:%f')
    
    print('Conversion coordonnees géographiques:')
    
    df['latitude'] =  df['coordonnees_geo'].str.strip().str.replace('[', ' ')\
        .str.replace(']', ' ').str.split(',', expand=True)[0].astype('float')
    
    df['longitude'] =  df['coordonnees_geo'].str.strip().str.replace('[', ' ')\
        .str.replace(']', ' ').str.split(',', expand=True)[1].astype('float')
    
    df = df.drop(columns='coordonnees_geo')
    
    print('Conversion bool')
    # Je passe les colonnes contenant des 0 et 1 au format booleen
    df['is_renting']     = df['is_renting'].str.strip().replace('NON', False)
    df['is_renting']     = df['is_renting'].str.strip().replace('OUI', True)
    df['is_renting']     = df['is_renting'].astype('bool')
    
    df['is_installed']   = df['is_installed'].str.strip().replace('NON', False)
    df['is_installed']   = df['is_installed'].str.strip().replace('OUI', True)
    df['is_installed']   = df['is_installed'].astype('bool')
    
    df['is_returning']   = df['is_returning'].str.strip().replace('NON', False)
    df['is_returning']   = df['is_returning'].str.strip().replace('OUI', True)
    df['is_returning']   = df['is_returning'].astype('bool')
    
    print("Conversion str")
    df['nom_arrondissement_communes'] = df['nom_arrondissement_communes'].str.strip().astype('string')
    df['name'] = df['name'].str.strip().astype('string')
   
    # station code contient des str, on les convertit en int
    print("Conversion int")
    
    # Si il y a des string dans stationcode
    if not((df['stationcode'].apply(type) == str).sum()==0):
        # Alors on les convertit en float
        df.loc[(df['stationcode'].apply(type) == str), 'stationcode'] = \
            df.loc[(df['stationcode'].apply(type) == str), 'stationcode'].str.extract('(\d+)')[0].astype(float)
    # On vire ensuite les NaN, sinon pas possbile de convertir en int
    df= df.dropna()
    df['stationcode'] = df['stationcode'].astype(int)
    
    # Idem pour capactiy
    if not((df['capacity'].apply(type) == str).sum()==0):
        df.loc[(df['capacity'].apply(type) == str), 'capacity'] = \
            df.loc[(df['capacity'].apply(type) == str), 'capacity'].str.extract('(\d+)')[0].astype(float)
    df= df.dropna()
    df['capacity'] = df['capacity'].astype(int)
    
    
    print("\n")

    return df



 # IMPORT DES JEUX DE DONNEES VELIB

# PROFILING

Nous sommes en présence de 14 fichiers plats csv contenant chacune une table dont les colonnes sont séparées par des ",".

Ce dataset contient les informations des stations Vélib dans les environs de Paris, mises à jour toutes les 10 minutes, et sur plusieurs mois (mai, juin, juillet, août, novembre et décembre) en 2022 et (janvier, février, mai, juin, juillet, août, septembre, octobre) en 2023. Ces données sont mises à disposition au public pour suivre l'évolution du service en temps réel.

Ces 14 fichiers sont constitués de 14 colonnes et plus de 6 millions de lignes avec une clé primaire correspondant à la colonne "station code", pour le reste des colonnes, on a :

- 'name' : nom de la station, format string texte,

- 'stationcode' : numéro unique d’identification de la station. Ce numéro identifie la station au sein du service Vélib’ Métropole, format int(entier),
   
- 'ebike' : nombre vélo electrique dispo au moment du ping, format int(entier),

- 'mechanical' : nombre vélo mécanique dispo au moment du ping, format string, 

- 'coordonnees_geo' : longitude de la station et latitude de la station, format string

- 'duedate': Date actualisation de la données (instant T), format string à convertir en format date

- 'numbikesavailable': nombre de vélos disponibles, format int,

- 'numdocksavailable': nombre de bornettes disponibles, format int,
  
- 'capacity' : nombre de bornettes dans la station, format int,
  
- 'is_renting': variable binaire indiquant si la station peut louer des vélos (is_renting=oui si le statut de la station est opérationnel), format string,
  
- 'is_installed' :  variable binaire indiquant si la station a déjà été déployée (oui) ou est encore en cours de déploiement (non), format string,
  
- -'is_returning': variable binaire indiquant si la station peut recevoir des vélos (is_renting=oui si le statut de la station est opérationnel), format string,
  
- 'ping_time' (# last_reported : date de la dernière mise-à-jour ), format string,

Dans la préparation qui suit, nous allons traiter chacune des tables pour sécuriser et nettoyer les jeux de données.    



# Traitement des données


In [8]:
# ATTENTION C'EST LONG A FAIRE TOURNER !

 
# On applique la fonction traitement_data sur chaque df et on enregistre le resultat sous forme de CSV.

# Repertoire ou sont stocké les données :
directory = 'Data\\Data_velib\\Data_raw'

i=0
# boucle qui parcours l'ensemble des fichiers du repertoire
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # On applique sur chaque fichier la fonction traitement_data, puis on sauvegarde les résultats dans
    # le repertoire Data\\Data_velib_traitées
    if os.path.isfile(f):
        print(i)
        print(filename+':')
        df = traitement_data(f)
        df.to_csv('Data\\Data_velib\\Data_traitées\\'+filename)
        
        print('\n')
        i+=1


0
Velib_AUG_2022.csv:
Chargement du csv ... 


C:\Users\avaun\AppData\Local\Temp\ipykernel_12368\3723746846.py:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, keep_default_na=False)




Traitement des données ... 


Conversion datetime :
Conversion coordonnees géographiques:
Conversion bool
Conversion str
Conversion int




1
Velib_AUG_2023.csv:
Chargement du csv ... 


C:\Users\avaun\AppData\Local\Temp\ipykernel_12368\3723746846.py:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, keep_default_na=False)




Traitement des données ... 


Conversion datetime :
Conversion coordonnees géographiques:
Conversion bool
Conversion str
Conversion int




KeyboardInterrupt: 